In [ ]:
parameters_path = ""
parameters_path = "../test_data/period-1/parameters.json"


In [ ]:
from rad import importer

import rad.praise.analysis as pr_analysis
import rad.praise.export as pr_export
import os
import pandas as pd
from IPython.display import Markdown as md
import plotly.graph_objects as go
import plotly.express as px

#load the files

#paths = ["../test_data/cross_period_exp/round-11/parameters.json","../test_data/cross_period_exp/round-12/parameters.json","../test_data/cross_period_exp/round-13/parameters.json","../test_data/cross_period_exp/round-14/parameters.json"]
#path = "../test_data/cross_period_exp"

#print(os.listdir("."))

categ_keywords = {'attendance':'join|attend|show up|participat','discussion':'question|ask|discuss|discussion','work':'help|work|design|make|write|hack|edit','lead':'host|lead|initiate|form|organize|steward','share':'share|spread','twitter':'twitter|tweet','hack':'hack|test','general':'support|awesome','IRL':'trip|conference'}


NUMBER_OF_PERIODS = 8 
STEP_SIZE = 'W'


#(rwdObj, rwdDist) = importer.load_multiple_periods(path, config={"mode":"root_folder"})
(rwdObj, rwdDist) = importer.load_multiple_periods(parameters_path)


#print(rwdDist)

#rwdDist['DATE'] = rwdDist.to_datetime(rwdDist['DATE'])

allrounds_distribution_data = pr_analysis.cross_period_helpers.combine_distribution_table(rwdDist["praise"])




allrounds_df = pr_analysis.cross_period_helpers.combine_quantification_data(rwdDist["praise"])



#print(allrounds_df)


# Cross-Period Analysis Report
This report aims to offer a perspective on the activity inside the praise system over several rounds.

In [ ]:
md(f"This report will cover <b>{NUMBER_OF_PERIODS}</b> weeks, divided into blocks of <b>{STEP_SIZE}</b> weeks each.")


# General Statistics
The full range will be subdivided into the following periods:

In [ ]:
end_date = allrounds_df["DATE"].max().ceil("D")



dates=pd.date_range(end=end_date, periods=NUMBER_OF_PERIODS, freq=STEP_SIZE)

print(dates)

In [ ]:
""" round_stats """

round_stats = pr_analysis.cross_period_round_stats.run(allrounds_df)

#print(round_stats)
#print(round_stats.iloc[0]["round_user_list"])
#print(type(round_stats.iloc[0]["round_user_list"]))

## Praise Involvement

### How much praise? 
This graph shows the trend of total number of praise instances across time.

In [ ]:
pr_analysis.cross_period_round_stats.printGraph(allrounds_df, _config={"x":"period_start_time", "y":"total_praise"})


### How many people give and receive praise?
Counting the unique ID of praise givers and receivers, we can visualize the change across time. In the figure, the blue line represents the amount of praise receivers and thered line the amount of givers.

In [ ]:
pr_analysis.cross_period_round_stats.printGraph(allrounds_df, _config={"x":"period_start_time", "y":["total_praise_receivers","total_praise_givers"]})


## Quantifier Involvement
Showing how many quantifiers are involved in each round.

In [ ]:
pr_analysis.cross_period_round_stats.printGraph(allrounds_df, _config={"x":"period_start_time", "y":"total_quantifiers"})

### Quantifier trend

### average score displacement: tendency to under/over-estimate?

In [ ]:
pr_analysis.cross_period_quantifier_stats.printGraph(allrounds_df,_config={"y":"pearson_coef"} )

### average score correlation coefficient: how much do i agree with other people?

In [ ]:
pr_analysis.cross_period_quantifier_stats.printGraph(allrounds_df,_config={"y":"av_score_displacement"} )

# System Health Evaluation

## Number of new TEC members involved in praise (either giving or receiving)
Counting the round-by-round change of unique IDs being either praise giver or praise receiver.

The blue line represents new IDs in this round, the red line represents IDs that are absent in this round but were present in the last round. The green line shows the net difference, with above 0 meaning more people joined praise than people left and below 0 meaning the opposite.

In [ ]:
pr_analysis.cross_period_round_stats.printGraph(allrounds_df, _config={"x":"period_start_time", "y":["round_user_new","round_user_left","round_net_user_diff"]})


## Distribution Equality

### Nakamoto Coefficient

The Nakamato Coefficient is defined as the smallest number of accounts who control at least 50% of the resource. Although its significance relates to the prospect of a 51% attack on a network, which may not be relevant in our context, we can still use it as an intuitive measure of how many individuals received the majority of rewards.

Bigger coefficient means more distributed (i.e. needs more people to pass 50%), smaller means more concentrated power. The number should always be an integer.

In [ ]:
# TODO: DOESNT FIT DIRECTLY INTO TIME-BASED ROUND STATS, REVIEW LATER
""" 
def nakamoto_coeff(x, key):
    value_sum= x[key].sum()
    x['PERCENTAGE'] = x[key] / value_sum
    sorted_x = x.sort_values(by='PERCENTAGE', ascending=False)
    tot_sum = np.array(sorted_x['PERCENTAGE'].cumsum())
    try:
        winner = np.array([k for k in range(len(tot_sum))
                          if tot_sum[k] > 0.5]).min() + 1
    except:
        winner = -1
    return winner
def nakamoto_coeff_ratio(x, key):
    winner = nakamoto_coeff(x, key)
    ratio = winner / len(x)
    return ratio """

""" round_stats['nakamoto']  = [nakamoto_coeff(allrounds_finaldist[round_name],'PRAISE REWARDS') for round_name in roundname_list]
round_stats['nakamoto_ratio']= [nakamoto_coeff_ratio(allrounds_finaldist[round_name],'PRAISE REWARDS') for round_name in roundname_list]
px.line(round_stats,x='period_start_time',y='nakamoto',markers=True,title='Minimum number of people receiving 50% of total rewards')

px.line(round_stats,x='period_start_time',y='nakamoto_ratio',markers=True,title='Ratio of people accumulating 50% of total rewards in relation to total number of receivers in that round') """


# Categorizing praise based on the praise reason

In [ ]:
#categ_praise_df,category_df = categorize_praise(master_allrounds,categ_keywords,save_csv=True)



##  the average, min, max score of each category


In [ ]:
#TODO Fix print
res = pr_analysis.cross_period_category_analysis.printDescription(allrounds_df, categ_keywords, _config={"mode":"summary-table", "num":"2"})
#print(res)

In [ ]:
#TODO Fix print

pr_analysis.cross_period_category_analysis.printGraph(allrounds_df, categ_keywords, _config={"mode":"avg-stats", "num":"3"})



## Top 3 highest scored praise in each category
A convenient way to check if the categorization keywords are reasonable.

In [ ]:
#TODO fix styling

pr_analysis.cross_period_category_analysis.printDescription(allrounds_df, categ_keywords, _config={"mode":"top-scored", "num":"3"})

## trend across time

In [ ]:
#test
#test_df = pr_analysis.cross_period_category_analysis.run(allrounds_df, categ_keywords)
#df = pr_analysis.cross_period_category_analysis.get_categ_stats(test_df, categ_keywords)
#df


In [ ]:
""" px.line(round_stats.filter(like='num'),title='number of praise in each category, across time') """

pr_analysis.cross_period_category_analysis.printGraph(allrounds_df, categ_keywords, _config={"mode":"trend", "y":"num"})



In [ ]:
""" px.line(round_stats.filter(like='_avg_score'),title='mean score of each category, across time') """

pr_analysis.cross_period_category_analysis.printGraph(allrounds_df, categ_keywords, _config={"mode":"trend", "y":"_avg_score"})
